# Download items from contentdm collection

In [ ]:
import requests
import pandas as pd

In [ ]:
base_url= 'https://cdm17173.contentdm.oclc.org'
collection = 'p17173coll38'
# item_id = df['']

def download_contentdm_image(base_url, collection, item_id, output_path):
    # Build manifest URL
    manifest_url = f"{base_url}/iiif/info/{collection}/{item_id}/manifest.json"
    print("Fetching manifest:", manifest_url)

    manifest = requests.get(manifest_url).json()

    # Each "canvas" is an image/page
    canvases = manifest["sequences"][0]["canvases"]

    for i, canvas in enumerate(canvases, start=1):
        # The IIIF image service ID is inside the "images" block
        service_id = canvas["images"][0]["resource"]["service"]["@id"]

        # Request the full image (100% size)
        img_url = f"{service_id}/full/pct:100/0/default.jpg"
        print(f"Downloading page {i}: {img_url}")

        img_data = requests.get(img_url).content
        filename = f"output_path/collection/{collection}_{item_id}_page{i}.jpg"
        with open(filename, "wb") as f:
            f.write(img_data)
        print("Saved:", filename)


# Example usage
download_contentdm_image(
    base_url="https://cdm17173.contentdm.oclc.org",
    collection="p17173coll1",
    item_id="1234",
    ouput_path="documents"
)
